In [ ]:
import pandas as pd
import numpy as np

In [ ]:
path = "C:\VScode_Projects\DP\datasets\CIC-IDS-2017\clean"
csv_file = path + "\CICIDS_converted_data_final_2.csv"
src_file = path + "\CICIDS_converted_data_with_attacks.csv"
chunk_size = 100_000

In [ ]:
# Initialize an empty list to store processed chunks
processed_chunks = []

# Read the file in chunks
for chunk in pd.read_csv(src_file, chunksize=chunk_size):
    chunk = chunk.dropna()
    
    # Append the processed chunk to the list
    processed_chunks.append(chunk)

# Combine all processed chunks into a single DataFrame
df = pd.concat(processed_chunks, ignore_index=True)

In [ ]:
df = df.drop_duplicates()

In [ ]:
df.info()

In [ ]:
df.columns

In [ ]:
df.srcip.value_counts()

In [ ]:
df.rename(columns={"protocol_m": "protocol"}, inplace=True)

In [ ]:
df.protocol.unique()

In [ ]:
df.protocol.value_counts()

In [ ]:
df = df[df["protocol"] != "other"]
df.protocol = df.protocol.apply(lambda x: 1 if x == "tcp" else 0 )

In [ ]:
df.protocol.value_counts()

In [ ]:
df.drop(columns=["label"], inplace=True)

In [ ]:
df.rename(columns={"attack_cat": "label"}, inplace=True)

In [ ]:
df.info()

In [ ]:
df.columns

In [ ]:
columns_to_move = ["protocol", "t_delta", "label", "srcip", "sport", "dstip", "dsport"]
for column_to_move in columns_to_move: 
    columns = [col for col in df.columns if col != column_to_move] + [column_to_move]
    df = df[columns]

In [ ]:
df.columns

In [ ]:
df.label.value_counts()

In [ ]:
# Separate BENIGN and other rows
benign_rows = df[df['label'] == 'BENIGN']
other_rows = df[df['label'] != 'BENIGN']

# Undersample BENIGN rows (e.g., randomly select 50% of them)
undersampled_benign = benign_rows.sample(frac=0.3, random_state=42)

# Combine undersampled BENIGN rows with the other rows
df = pd.concat([undersampled_benign, other_rows])

In [ ]:
df.label.value_counts()

In [ ]:
df.to_csv(csv_file, index=False)